# **1 Dataset-Preprocessing** 


In [1]:
import urllib.parse

normal_file_raw = '/Users/yalinghu/Downloads/normalTrafficTraining.txt'
anomalous_file_raw = '/Users/yalinghu/Downloads/anomalousTrafficTest.txt'

normal_file_pre = 'normal.txt'
anomalous_file_pre = 'anomalous.txt'


def pre_file(file_in, file_out=None):
    with open(file_in, 'r', encoding='utf-8') as f_in:
        lines = f_in.readlines()
    res = []
    for i in range(len(lines)):
        line = lines[i].strip()
        if line.startswith("GET"):
            res.append("GET " + line.split(" ")[1])
        elif line.startswith("POST") or line.startswith("PUT"):
            method = line.split(' ')[0]
            url = line.split(' ')[1]
            j = 1
            while True:
                if lines[i + j].startswith("Content-Length"):
                    break
                j += 1
            j += 2
            data = lines[i + j].strip()
            url += '?'+data
            res.append(method + ' ' + url)

    with open(file_out, 'w', encoding='utf-8') as f_out:
        for line in res:
            line = urllib.parse.unquote(line, encoding='ascii', errors='ignore').replace('\n', '').lower()
            f_out.writelines(line + '\n')


if __name__ == '__main__':
    pre_file(normal_file_raw, normal_file_pre)
    pre_file(anomalous_file_raw, anomalous_file_pre)

normal.txt数据预提取完成 36000条记录
anomalous.txt数据预提取完成 25065条记录


# **2 Feature Extraction**



## 2.1 TF-IDF



In [3]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = f.readlines()
    result = []
    for d in data:
        d = d.strip()
        if len(d) > 0:
            result.append(d)
    return result

normal_requests = load_data('normal.txt')
anomalous_requests = load_data('anomalous.txt')

all_requests = normal_requests + anomalous_requests
y_normal = [0] * len(normal_requests)
y_anomalous = [1] * len(anomalous_requests)
y = y_normal + y_anomalous

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="word", sublinear_tf=True)
X = vectorizer.fit_transform(all_requests)

In [9]:
X

array([[1.18000000e+02, 1.00000000e+00, 6.50000000e+01, ...,
        1.87500000e-01, 7.18750000e-01, 9.37500000e-02],
       [1.19000000e+02, 2.00000000e+00, 6.50000000e+01, ...,
        1.87500000e-01, 7.18750000e-01, 9.37500000e-02],
       [1.17000000e+02, 1.00000000e+00, 6.00000000e+01, ...,
        6.89655172e-02, 9.31034483e-01, 0.00000000e+00],
       ...,
       [2.96000000e+02, 2.00000000e+00, 2.41000000e+02, ...,
        2.20689655e-01, 6.89655172e-01, 8.96551724e-02],
       [2.96000000e+02, 1.00000000e+00, 2.42000000e+02, ...,
        2.20689655e-01, 7.03448276e-01, 7.58620690e-02],
       [2.97000000e+02, 2.00000000e+00, 2.42000000e+02, ...,
        2.20689655e-01, 7.03448276e-01, 7.58620690e-02]])

In [4]:
vectorizer.vocabulary_

{'get': 19606,
 'http': 20938,
 'localhost': 23232,
 '8080': 8750,
 'tienda1': 31274,
 'index': 21282,
 'jsp': 21914,
 'publico': 27735,
 'anadir': 11513,
 'id': 21106,
 'nombre': 25744,
 'vino': 32658,
 'rioja': 28665,
 'precio': 27578,
 '100': 1079,
 'cantidad': 14443,
 '55': 6608,
 'b1': 12410,
 'aadir': 10396,
 'al': 10904,
 'carrito': 14650,
 'post': 27513,
 'autenticar': 12288,
 'modo': 24778,
 'entrar': 17935,
 'login': 23252,
 'choong': 15302,
 'pwd': 27837,
 'd1se3cin': 16347,
 'remember': 28400,
 'off': 25987,
 'caracteristicas': 14510,
 'errormsg': 18064,
 'credenciales': 16098,
 'incorrectas': 21263,
 'miembros': 24577,
 'pagar': 26485,
 'insertar': 21368,
 '2672': 3274,
 'pasar': 26717,
 'por': 27465,
 'caja': 14192,
 'productos': 27660,
 'registro': 28345,
 'cen': 14990,
 'password': 26740,
 '40a5e': 4953,
 'nurit': 25864,
 'apellidos': 11723,
 'ferrandez': 18694,
 'cabaas': 14099,
 'email': 17719,
 'tarpey': 30952,
 'bwds': 13986,
 'ne': 25477,
 'dni': 17096,
 '17621392b

In [5]:
X.shape

(61065, 33550)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

## Model: 1. K-nearest neighbors (KNN) ##

In [15]:
%%time

# from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import StandardScaler


standardScalar = StandardScaler(with_mean=False)
standardScalar.fit(X_train)
X_train = standardScalar.transform(X_train)
X_test = standardScalar.transform(X_test)

# # grid search
# param_grid = [
#     {
#         'weights': ['uniform'],
#         'n_neighbors': [i for i in range(2, 11)] 
#     },
#     {
#         'weights': ['distance'],
#         'n_neighbors': [i for i in range(2, 11)],
#         'p': [i for i in range(1, 6)]
#     }
# ]

#grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, n_jobs=-1, cv=5)
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

CPU times: user 56.9 ms, sys: 7.33 ms, total: 64.3 ms
Wall time: 64 ms


KNeighborsClassifier()

In [16]:
knn_clf.score(X_test, y_test)

0.8695713281799016

In [17]:
y_predict = knn_clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.8722124780756703
0.8927930238522698
0.8823827629911281


## 2. Logistic Regression ##

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'C': [0.1, 1, 3, 5, 7],
        'penalty': ['l1', 'l2']
    }
]

grid_search = GridSearchCV(LogisticRegression(), param_grid, n_jobs=-1, cv=5)


In [8]:
%%time
grid_search.fit(X_train, y_train)

/Users/yalinghu/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.87894054        nan 0.94900926        nan 0.96307214
        nan 0.96710473        nan 0.96757553]
  warnings.warn(


CPU times: user 3.58 s, sys: 178 ms, total: 3.76 s
Wall time: 7.51 s


/Users/yalinghu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 3, 5, 7], 'penalty': ['l1', 'l2']}])

In [9]:
grid_search.best_score_

0.9675755329282116

In [10]:
grid_search.best_params_

{'C': 7, 'penalty': 'l2'}

In [11]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.9734708916728076

In [12]:
y_predict = best_knn_clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.9922763355503111
0.9413800122124975
0.9661583455191143


## 3. Decision Tree ##

In [13]:
from sklearn.tree import DecisionTreeClassifier

param_grid = [
    {
        'max_depth':[i for i in range(1, 10)],
        'min_samples_leaf':[i for i in range(1, 20)],
        'min_samples_split':[i for i in range(10, 30)],
    }
]

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, n_jobs=-1, cv=5)

In [14]:
%%time
grid_search.fit(X_train, y_train)

CPU times: user 59.7 s, sys: 9.96 s, total: 1min 9s
Wall time: 52min 32s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                          'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                               11, 12, 13, 14, 15, 16, 17, 18,
                                               19],
                          'min_samples_split': [10, 11, 12, 13, 14, 15, 16, 17,
                                                18, 19, 20, 21, 22, 23, 24, 25,
                                                26, 27, 28, 29]}])

In [15]:
grid_search.best_score_

0.8979776073725991

In [16]:
grid_search.best_params_

{'max_depth': 9, 'min_samples_leaf': 19, 'min_samples_split': 10}

In [17]:
best_tree_clf = grid_search.best_estimator_
best_tree_clf.score(X_test, y_test)

0.90084336362892

In [18]:
y_predict = best_tree_clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.951904296875
0.7936087929981681
0.8655788655788657


## 4. SVM ##

In [19]:
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

standardScalar = StandardScaler(with_mean=False)
standardScalar.fit(X_train)
X_train = standardScalar.transform(X_train)
X_test = standardScalar.transform(X_test)

In [20]:
%%time
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train, y_train)

CPU times: user 7min 2s, sys: 1.57 s, total: 7min 3s
Wall time: 7min 5s


SVC()

In [21]:
svm_clf.score(X_train, y_train)

0.9746376811594203

In [22]:
svm_clf.score(X_test, y_test)

0.9619258167526407

In [23]:
y_predict = svm_clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.9625623960066556
0.941990637085284
0.952165415080753


## 5. Random Forest ##

In [24]:
from sklearn.ensemble  import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=500,
                               random_state=666,
                               oob_score=True,
                               n_jobs=-1)

In [25]:
%%time
rf_clf.fit(X_train, y_train)

CPU times: user 14min 33s, sys: 2.17 s, total: 14min 35s
Wall time: 1min 57s


RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True,
                       random_state=666)

In [26]:
rf_clf.score(X_test, y_test)

0.9643822156718251

In [27]:
y_predict = rf_clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.9612690564482901
0.949725218807246
0.9554622709122554


## 2.2 Vectors of custom variables

“longitud de la URL”, “tipo de solicitud“, “longitud de la sección de parámetros“, “número de parámetros“, “longitud máxima del parámetro“, “cantidad de números en el parámetro“, “proporción del valor del parámetro que es numérico“, “proporción de letras en el parámetro“, “número de caracteres especiales, proporción de caracteres especiales“



In [13]:
def vector(data):
    feature = []

    for i in range(len(data)):
        s = data[i].split("?", 1)
        if len(s) != 1:

            url_len = len(data[i])

            method = data[i].split(" ")[0]
            if method == "get":
                url_method = 1
            elif method == "post":
                url_method = 2
            else:
                url_method = 3

            query = s[1]
            parameter_len = len(query)
            parameters = query.split("&")
            parameter_num = len(parameters)

            parameter_max_len = 0
            parameter_number_num = 0
            parameter_str_num = 0
            parameter_spe_num = 0
            par_val_sum = 0

            for parameter in parameters:
                try:
                    [par_name, par_val] = parameter.split("=", 1)
                except ValueError as err:
                    print(err)
                    print(data[i])
                    break

                par_val_sum += len(par_val)
                if parameter_max_len < len(par_val):
                    parameter_max_len = len(par_val)
                parameter_number_num += len(re.findall("\d", par_val))
                parameter_str_num += len(re.findall(r"[a-zA-Z]", par_val))
                parameter_spe_num += len(par_val) - len(re.findall("\d", par_val)) - len(
                    re.findall(r"[a-zA-Z]", par_val))

            try:
                parameter_number_rt = parameter_number_num / par_val_sum
                parameter_str_rt = parameter_str_num / par_val_sum
                parameter_spe_rt = parameter_spe_num / par_val_sum
                feature.append([url_len, url_method, parameter_len, parameter_num,
                                parameter_max_len, parameter_number_num, parameter_str_num,
                                parameter_spe_num, parameter_number_rt, parameter_str_rt,
                                parameter_spe_rt])
            except ZeroDivisionError as err:
                print(err)
                print(data[i])
                continue
    return feature

In [17]:
import pandas as pd
log_iti=pd.read_csv("PD_traffic_dataset.csv")

In [31]:
df = pd.read_csv ('PD_traffic_dataset.csv')
data= df.values.tolist()

In [32]:
data

[[25011,
  'PUT',
  'http://localhost:8080/tienda1/publico/pagar.jsp',
  'HTTP/1.1',
  'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)',
  'no-cache',
  'no-cache',
  'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5',
  'x-gzip, x-deflate, gzip, deflate',
  'utf-8, utf-8;q=0.5, *;q=0.5',
  'en',
  'localhost:9090',
  'close',
  43.0,
  'application/x-www-form-urlencoded',
  'JSESSIONID=71175AB8199AED8AA8425802422F03DD',
  'precio=8157',
  'anom'],
 [25011,
  'PUT',
  'http://localhost:8080/tienda1/publico/pagar.jsp',
  'HTTP/1.1',
  'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)',
  'no-cache',
  'no-cache',
  'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5',
  'x-gzip, x-deflate, gzip, deflate',
  'utf-8, utf-8;q=0.5, *;q=0.5',
  'en',
  'localhost:9090',
  'close',
  43.0,
  'application/x-www-form-urlencoded',
  'JSESSIONID=71175

In [7]:
# labelling
import numpy as np

normal = np.loadtxt("vector_normal.txt")
anomalous = np.loadtxt("vector_anomalous.txt")

all_requests = np.concatenate([normal, anomalous])
X = all_requests

y_normal = np.zeros(shape=(normal.shape[0]), dtype='int')
y_anomalous = np.ones(shape=(anomalous.shape[0]), dtype='int')
y = np.concatenate([y_normal, y_anomalous])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

## Model: 1. KNN

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

standardScalar = StandardScaler()
standardScalar.fit(X_train)
X_train = standardScalar.transform(X_train)
X_test_std = standardScalar.transform(X_test)

param_grid = [
    {
        'weights': ['uniform'],
        'n_neighbors': [i for i in range(2, 11)] 
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)],
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, n_jobs=-1, cv=5)

# pipe_grid_knn = Pipeline([
#     ("sta_scaler", StandardScaler()),
#     ("grid_sea", grid_search)
# ])

In [31]:
%%time
grid_search.fit(X_train, y_train)

CPU times: user 502 ms, sys: 254 ms, total: 756 ms
Wall time: 34 s


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}])

In [32]:
grid_search.best_score_

0.8759617181052846

In [33]:
grid_search.best_params_

{'n_neighbors': 10, 'p': 3, 'weights': 'distance'}

In [34]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test_std, y_test)

0.8733661278988053

In [35]:
from sklearn.metrics import confusion_matrix

y_predict = best_knn_clf.predict(X_test_std)
confusion_matrix(y_test, y_predict)

array([[2735,  481],
       [ 420, 3479]])

In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.8785353535353535
0.8922800718132855
0.8853543707850872


In [37]:
X_test = X_test_std

## 2. Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression

param_grid = [
    {
        'C': [0.1, 1, 3, 5, 7],
        'penalty': ['l1', 'l2']
    }
]

grid_search = GridSearchCV(LogisticRegression(), param_grid, n_jobs=-1, cv=5)


In [39]:
%%time
grid_search.fit(X_train, y_train)

CPU times: user 698 ms, sys: 30.8 ms, total: 729 ms
Wall time: 1.33 s


/Users/yalinghu/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.66481615        nan 0.67303809        nan 0.67497134
        nan 0.67999595        nan 0.68340426]
  warnings.warn(
/Users/yalinghu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 3, 5, 7], 'penalty': ['l1', 'l2']}])

In [40]:
grid_search.best_score_

0.6834042618611107

In [41]:
grid_search.best_params_

{'C': 7, 'penalty': 'l2'}

In [42]:
best_log_clf = grid_search.best_estimator_
best_log_clf.score(X_test, y_test)

0.692340126493324

In [43]:
y_predict = best_log_clf.predict(X_test_std)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.745830937320299
0.6652987945627084
0.7032669106682934


## 3. Decision Tree

In [44]:
from sklearn.tree import DecisionTreeClassifier

param_grid = [
    {
        'max_depth':[i for i in range(1, 10)],
        'min_samples_leaf':[i for i in range(1, 20)],
        'min_samples_split':[i for i in range(10, 30)],
    }
]

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, n_jobs=-1, cv=5)


In [45]:
%%time
grid_search.fit(X_train, y_train)

CPU times: user 11.6 s, sys: 1.69 s, total: 13.3 s
Wall time: 2min 10s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                          'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                               11, 12, 13, 14, 15, 16, 17, 18,
                                               19],
                          'min_samples_split': [10, 11, 12, 13, 14, 15, 16, 17,
                                                18, 19, 20, 21, 22, 23, 24, 25,
                                                26, 27, 28, 29]}])

In [46]:
grid_search.best_score_

0.7973578012057603

In [47]:
grid_search.best_params_

{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 28}

In [48]:
best_tree_clf = grid_search.best_estimator_
best_tree_clf.score(X_test, y_test)

0.8042164441321152

In [49]:
y_predict = best_tree_clf.predict(X_test_std)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.7658039881204921
0.9258784303667608
0.8382677348194589


In [50]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [51]:
tree_clf = DecisionTreeClassifier(max_depth=9, min_samples_leaf=1, min_samples_split=27)

In [52]:
tree_clf.fit(X_train_raw, y_train)

DecisionTreeClassifier(max_depth=9, min_samples_split=27)

In [53]:
tree_clf.score(X_train_raw, y_train)

0.80965599634562

In [54]:
tree_clf.score(X_test_raw, y_test)

0.8042164441321152

In [55]:
y_predict = best_tree_clf.predict(X_test_raw)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.5479971890372453
1.0
0.708007989831124


## 4. SVM

In [56]:
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)
standardScalar = StandardScaler()
standardScalar.fit(X_train)
X_train = standardScalar.transform(X_train)
X_test = standardScalar.transform(X_test)

In [57]:
from sklearn.svm import SVC

param_grid = [
    {
        'kernel': ["poly"],
        'degree': [1, 2, 3],
        'C': [0.1, 1, 3, 5]
    }
]

grid_search = GridSearchCV(SVC(), param_grid, n_jobs=-1, cv=5)

In [58]:
%%time
grid_search.fit(X_train, y_train)

CPU times: user 21.8 s, sys: 182 ms, total: 22 s
Wall time: 6min


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 3, 5], 'degree': [1, 2, 3],
                          'kernel': ['poly']}])

In [59]:
grid_search.best_params_

{'C': 5, 'degree': 3, 'kernel': 'poly'}

In [60]:
grid_search.best_score_

0.7021678272198845

In [61]:
best_svm_clf = grid_search.best_estimator_
best_svm_clf.score(X_test, y_test)

0.7114546732255798

In [62]:
y_predict = best_svm_clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.7404898384575299
0.7289048473967684
0.7346516737753651


## 5. Random Forest

In [63]:
from sklearn.ensemble  import RandomForestClassifier

In [64]:
rf_clf = RandomForestClassifier(n_estimators=500,
                               random_state=666,
                               oob_score=True,
                               n_jobs=-1)

In [65]:
%%time
rf_clf.fit(X_train, y_train)

CPU times: user 15.4 s, sys: 414 ms, total: 15.8 s
Wall time: 3.34 s


RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True,
                       random_state=666)

In [66]:
rf_clf.score(X_test, y_test)

0.9460295151089249

In [67]:
y_predict = rf_clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score

print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))
print(f1_score(y_test, y_predict))

0.9461792333079462
0.9558861246473455
0.9510079101811687
